In [1]:
import datetime
import pandas as pd
import requests
import io
import json

url = 'http://gis.sutran.gob.pe/alerta_sutran/script_cgm/carga_xlsx.php'
headers = {'Accept': '/',
'Accept-Encoding': 'gzip, deflate',
'Accept-Language': 'en,es;q=0.9,en-GB;q=0.8',
'Connection': 'keep-alive',
'Content-Length': '9',
'Content-type': 'application/x-www-form-urlencoded; charset=UTF-8-SIG',
'Host': 'gis.sutran.gob.pe',
'Origin': 'http://gis.sutran.gob.pe',
'Referer': 'http://gis.sutran.gob.pe/alerta_sutran/',
'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36'}
body = {'tipo':'MAPA'}
req = requests.post(url, headers=headers, data=body)
now = datetime.datetime.now()

my_json = json.load(io.BytesIO(req.content))

restringido = {}
restringido['data'] = my_json['restringido']

interrumpido = {}
interrumpido['data'] = my_json['interrumpido']

with open('restringido.json', 'w') as f:
    json.dump(restringido, f)

with open('interrumpido.json', 'w') as f:
    json.dump(interrumpido, f)

df_r = pd.read_json('restringido.json', encoding = 'utf-8-sig', orient='split')
df_restringido = pd.json_normalize(df_r['properties'])

df_i = pd.read_json('interrumpido.json', encoding = 'utf-8-sig', orient='split')
df_interrumpido = pd.json_normalize(df_i['properties'])

frames = [df_restringido, df_interrumpido]
df_total = pd.concat(frames, ignore_index=True)
df_total = df_total[df_total['motivo'] == 'HUMANO']
df_total = df_total.replace('TRANSITO', 'TRÁNSITO', regex=True)
df_total = df_total.replace('"', '', regex=True)
df_total['style'] = df_total['estado'].str.split(' ').str[1].str.lower()
df_total.to_csv(f'{now}.csv', index=False)
df_total

,item,visualizacion_,tipo_alerta_,estado,fecha_evento,fecha_actualizacion,ubigeo,afectacion,evento,latitud,...,_sort_,_fecha_actualizacion_,cant_vehiculos_detenidos_pasajeros,cant_vehiculos_detenidos_mercancias,fuente,motivo,codigo_via,nombre_carretera,alerta_activa_24_horas,style
0,211,SI,VIGENTE,TRÁNSITO RESTRINGIDO,08/01/2023,18/01/2023 10:45 HORAS,MADRE DE DIOS/TAMBOPATA/TAMBOPATA,KM 402 Y KM 409,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-12.6898325,...,-12.6898325+-69.4042951,2023 10:45 HORAS+01+18+10:45,0,0,CEOPOL,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,restringido
3,203,SI,VIGENTE,TRÁNSITO RESTRINGIDO,13/01/2023,18/01/2023 10:45 HORAS,MADRE DE DIOS/TAMBOPATA/INAMBARI,KM 265,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-13.065658,...,-13.065658+-70.3524946,2023 10:45 HORAS+01+18+10:45,0,0,CEOPOL,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,restringido
5,204,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,04/01/2023,18/01/2023 10:45 HORAS,MADRE DE DIOS/TAMBOPATA/LABERINTO,KM 382+200,GRUPO DE PERSONAS PROTESTAN SOLICITANDO NUEVA...,-11.4068843,...,-11.4068843+-69.4784365,2023 10:45 HORAS+01+18+10:45,2,4,IIRSA SUR / UD MADRE DE DIOS,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,interrumpido
6,199,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,14/01/2023,18/01/2023 10:45 HORAS,MADRE DE DIOS/TAHUAMANU/TAHUAMANU,KM 548+950,GRUPO DE PERSONAS PROTESTAN SOLICITANDO NUEVA...,-11.665167,...,-11.665167+-69.229412,2023 10:45 HORAS+01+18+10:45,0,0,IIRSA SUR / UD MADRE DE DIOS,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,interrumpido
7,209,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,06/01/2023,18/01/2023 10:45 HORAS,MADRE DE DIOS/TAMBOPATA/LAS PIEDRAS,KM 511+800,(PUENTE MAVILA) PARO INDEFINIDO SOLICITANDO NU...,-11.931332,...,-11.931332+-69.11748,2023 10:45 HORAS+01+18+10:45,0,0,CEOPOL,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,interrumpido
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,188,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,17/01/2022,18/01/2023 10:45 HORAS,LA LIBERTAD/TRUJILLO/LAREDO,KM 25,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-8.032668333,...,-8.032668333+-78.84158167,2023 10:45 HORAS+01+18+10:45,1,3,CEOPOL,HUMANO,PE-10A,CARRETERA TRUJILLO - AGALLPAMPA - QUIRUVILCA,SI,interrumpido
105,191,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,11/12/2022,18/01/2023 10:45 HORAS,LA LIBERTAD/VIRU/VIRU,KM 528,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-8.382992,...,-8.382992+-78.855957,2023 10:45 HORAS+01+18+10:45,0,1,CONCESIONARIA AUTOPISTA DEL NORTE,HUMANO,PE-1N,LONGITUDINAL DE LA COSTA NORTE,SI,interrumpido
106,189,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,16/01/2023,18/01/2023 10:45 HORAS,LA LIBERTAD/VIRU/CHAO,KM 500,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-8.546018905,...,-8.546018905+-78.67488073,2023 10:45 HORAS+01+18+10:45,15,8,DESPRCAR VIRU,HUMANO,PE-1N,LONGITUDINAL DE LA COSTA NORTE,SI,interrumpido
107,10,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,18/01/2023,18/01/2023 10:45 HORAS,ANCASH/RECUAY/CATAC,KM 543,GRUPO DE PERSONAS PROTESTAN SOLICITANDO NUEVA...,-9.85239,...,-9.85239+-77.406543,2023 10:45 HORAS+01+18+10:45,0,0,UD. HUARAZ,HUMANO,PE-3N,LONGITUDINAL DE LA SIERRA NORTE,SI,interrumpido
